In [58]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144
import pandas as pd
import numpy as np
import geopandas as gpd
import dill, urllib2, zipfile, os
pd.options.mode.use_inf_as_na = True
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Solar PV data
The "openpv_all.zip" was downloaded from "https://openpv.nrel.gov/search." 

In [3]:
#Download from the site
pv_url = "https://maps-api.nrel.gov/open_pv/installs/download_all"
zip_pv = urllib2.urlopen(pv_url)
pv_content = zip_pv.read()

In [4]:
with open('openpv_all.zip', 'wb') as f:
    f.write(pv_content)

In [5]:
# Unzip
zip_file = zipfile.ZipFile('openpv_all.zip', 'r')
zip_file.extractall()
zip_file.close()

In [6]:
#Load data(takes a few seconds) and create copy to work from
solar_load = pd.read_csv('openpv_all.csv', parse_dates=[1], infer_datetime_format=True)
solar2 = solar_load

C:\Users\Sarah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (2,5,8,12,13,14,15,16,20,23,25,26,27,28,32,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#Standardize "residential" category, then limit to those installs 
solar2['install_type']= solar2['install_type'].str.replace('/SF','')
solar2['install_type']= solar2['install_type'].str.replace('Residential','residential')
solar2 = solar2[solar2['install_type'] == 'residential']

In [8]:
#Create state-county key for future merges
solar2['st_cnty']= solar2['state']+'-'+solar2['county'].str.upper()

C:\Users\Sarah\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
#Not useful for analysis but fun - check the number of references to different solar incentive programs.
#Note that this information is extremely biased - programs report their own data. 
#There are probably some programs that are very consistent in reporting, and many that are not. 
#solar2.incentive_prog_names.value_counts()

The BLSSeriesReport was modified in Excel based on a download from https://data.bls.gov/cgi-bin/surveymost?cu. 
The columns are year, annual CPI (all items less food and energy), a re-index to the year 2000, and the decimal of  the re-index. 

In [10]:
cpi = pd.read_excel('BLSSeriesReport.xlsx', sheet_name=0)

In [11]:
#Pull out year and use to merge with CPI, then create deflated measures (to year 2000) of cost and cost per watt
solar2['Year'] = solar2['date_installed'].dt.year
solar2 = solar2.merge(cpi, on='Year', how='left')
solar2['rl_cost'] = solar2.cost/solar2.Decimal
solar2['rl_cost_per_watt'] = solar2.cost_per_watt/solar2.Decimal

C:\Users\Sarah\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
#restrict solar2 to useful columns
solar2 = solar2[['state',
 'date_installed',
 'incentive_prog_names',
 'type',
 'size_kw',
 'zipcode',
 'install_type',
 'installer',
 'cost_per_watt',
 'cost',
 'utility_clean',
 'county',
 'annual_PV_prod',
 'annual_insolation',
 'rebate',
 'st_cnty',
 'Year',
 u'Decimal',
 'rl_cost',
 'rl_cost_per_watt']]

In [166]:
# top 5 solar states
solar_state = solar2.groupby('state').agg({'type':'count', 'size_kw':'sum'})
totalpv = solar_state.sort_values('type', ascending=False).head(20).reset_index()
totalpv

,state,size_kw,type
0,CA,4190938.50,574312
1,AZ,605212.52,85498
2,MA,468208.38,64845
3,NY,324725.23,43956
4,NJ,353618.84,43196
5,CT,114263.30,15493
6,NV,96792.21,14608
7,TX,103359.36,12414
8,MD,69316.34,10702
9,PA,74049.88,9749


In [13]:
#Save the somewhat-cleaned up data, including a time-series version
solar2.to_pickle('solar.pkl')
pvdf_time = solar2[['date_installed','state', 'type', 'rl_cost_per_watt']].set_index('date_installed').sort_index()
pvdf_time = pvdf_time.loc['2002-01-01':'2015-12-31']
pvdf_time.to_pickle('solar_ts.pkl')

In [14]:
solar2 = pd.read_pickle('solar.pkl')

### Electricity prices
County-based electricity prices must be calculated from prices reported by individual utilities based on those utilities' service areas. These calculations are still imperfect; they only find the average price over all utilities in a given county. Weighting prices by number of customers in a county is impossible because the service area data has no customer breakdown. 

In [16]:
#Create column names for import of utility price/customer data
salescolnames=['Data Year',
 'Utility Number',
 'Utility Name',
 'Part',
 'Service Type',
 'Data Type\nO = Observed\nI = Imputed',
 'State',
 'Ownership',
 'BA_CODE',
 'RES Revenue Thousand Dollars',
 'RES Megawatthours',
 'RES Customer Count',]

The files below were downloaded from https://www.eia.gov/electricity/data/eia861/index.html.
The csv files were manually extracted into their own folder due to variations in file names.

In [17]:
# read in utility sales data from 2008 to 2015 to get prices
append_data = []
for WorkingFile in os.listdir('utilitydata/Sales'):
    print "working on file " + WorkingFile
    dfxls = pd.read_excel('utilitydata/Sales/'+WorkingFile, names = salescolnames, thousands = ',', skiprows=3, usecols = list(range(0,12)))
    dfxls.drop(dfxls[dfxls['Utility Number']>80000].index, inplace=True) #drop missing utilities
    dfxls.drop(dfxls.tail(1).index, inplace=True) #drop informational row
    append_data.append(dfxls)
merged = pd.concat(append_data)

working on file file2_2008.xls
working on file file2_2009.xls
working on file file2_2010.xls
working on file file2_2011.xls
working on file retail_sales_2012.xls
working on file Sales_Ult_Cust_2013.xls
working on file Sales_Ult_Cust_2014.xls
working on file Sales_Ult_Cust_2015.xlsx


In [18]:
#convert columns from object to numeric and restrict to those columns
util_sales = merged

util_sales['RESrev'] = util_sales['RES Revenue Thousand Dollars'].apply(pd.to_numeric, errors='coerce')
util_sales['RESmwh'] = util_sales['RES Megawatthours'].apply(pd.to_numeric, errors = 'coerce')
util_sales['util_number'] = util_sales['Utility Number'].astype(int)
util_sales['Year'] = util_sales['Data Year'].astype(int)
util_sales['ut_st']=util_sales['util_number'].map(str) + util_sales['State'] #create merge key

util_sales = util_sales[['Year', 'RESrev', 'RESmwh', 'ut_st']]

In [20]:
util_sales['price'] = util_sales['RESrev']/util_sales['RESmwh']

#2008-2012 prices are in cents, post-2012 prices are in dollars 
def standard_price(price):
    return price*100
util_sales.loc[util_sales['Year']>2012, 'price'] = util_sales[util_sales['Year']>2012]['price'].apply(lambda row: standard_price(row))

C:\Users\Sarah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Sarah\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
# import utility service territory 
utilityterritory = pd.read_excel('utilitydata/Service_Territory_2015.xlsx')
utilityterritory['ut_st'] = utilityterritory['Utility Number'].map(str) + utilityterritory['State']

In [22]:
# merge with utility price data
utility = pd.merge(util_sales, utilityterritory,  on='ut_st', how='left')
utility = utility[['Year', 'RESrev', 'RESmwh', 'Utility Name', 'State', 'County', 'price']].dropna(how='any')
utility['st_cnty'] = utility['State'].str.upper() +'-'+ utility['County'].str.upper()

In [24]:
#group and aggregate utility data by county and year and save cleaned data
county_price = utility.groupby(['Year','st_cnty'])
counties = county_price.agg({'price': np.mean, 'State': lambda y: pd.Series.unique(y),'Utility Name': lambda x: ','.join(sorted(pd.Series.unique(x)))}).reset_index()
counties = counties.rename(index=str, columns={'Data Year_x':'Year'})

counties.to_pickle('counties.pkl')

### Suitability of small buildings
The below CSV was manually downloaded from the NSRDB viewer (https://nsrdb.nrel.gov/nsrdb-viewer). For each county, it reports the number of small buildings (nbld) in that county and the percentage of those buildings that are estimated to be suitable for solar installations (pct_suitab).

In [25]:
#import lidar national small suitability data
lidar = pd.read_csv('national_small_suitability.csv', na_values=-999)
lidar['st_cnty'] = lidar['state'].str.strip() + '-' + lidar['county'].str.strip()
lidar_county = lidar.drop(lidar.columns[[0,1,3,4,5,6,7,8,9,10]], axis =1)
lidar_groups = lidar_county.groupby('st_cnty')
lidar_count = lidar_groups.agg({'nbld': np.sum, 'pct_suitab': np.mean, 'fipsstco': lambda x: pd.Series.unique(x)})
lidar_count = lidar_count.reset_index()

lidar_count.to_pickle('lidar_cnty.pkl')

In [26]:
counties = pd.read_pickle('counties.pkl')
lidar_count = pd.read_pickle('lidar_cnty.pkl')

Electricity prices have a different CPI, also downloaded from https://data.bls.gov/cgi-bin/surveymost?cu (series ID CUUR0000SEHF01). As before, this CPI Excel spreadsheet was downloaded then modified to re-index to the year 2000. 

In [27]:
el_cpi = pd.read_excel('BLSSeriesReport_Electricity.xlsx', sheet_name=0)

In [29]:
#Merge county electricity price with small building suitability and merge with CPI to get real prices
suitability = pd.merge(counties, lidar_count, on='st_cnty', how='left')
suitability = suitability.dropna(how='any')
suitability = suitability.merge(el_cpi, on='Year', how='left')
suitability['rl_price'] = suitability.price/suitability.Decimal
suitability = suitability[['Year',
 'st_cnty',
 'price',
 'State',
 'Utility Name',
 'pct_suitab',
 'fipsstco',
 'nbld',
 'rl_price']]

In [32]:
#suitability now represents lidar and real price data
suitability.to_pickle('suitability.pkl')

In [33]:
suitability = pd.read_pickle('suitability.pkl')

In [34]:
#group and aggregate suitability to average price over 2008 - 2012
suitability_agg = suitability.groupby('st_cnty').agg({'State': lambda x: pd.Series.unique(x), 
                                                      'pct_suitab': lambda x: pd.Series.unique(x),
                                                      'fipsstco': lambda x: pd.Series.unique(x),
                                                      'nbld': lambda x: pd.Series.unique(x),
                                                      'rl_price': np.mean}).reset_index()

### Election data
The CSV below is from https://github.com/helloworlddata/us-presidential-election-county-results, compiled by Dan Nguyen. 

In [36]:
#Load data on 2004-2012 presidential election votes, by county
elec = pd.read_csv('us_presidential_election_county_results_2004_2012.csv')
elec = elec[['year', 'state', 'county', 'fips', 'pct_rep', 'pct_dem', 'pct_oth']].rename({'fips':'fipsstco'}, axis=1)
elec = elec[elec.state != 'AK']
elec['fipsstco'] = elec.fipsstco.astype(str).astype(float)

In [37]:
#standardize county names to single names (i.e. not "___ County")
elec['county'].replace(regex=True, inplace=True, to_replace=r' County', value=r'')
elec['year']= elec.year.astype(str)

In [38]:
#reshape to columns for each year's results
elec_melt = elec.set_index(['state', 'county', 'fipsstco', 'year']).unstack('year')
elec_melt.columns = ["_".join(pair) for pair in elec_melt.columns]
elec_melt.reset_index(inplace=True)
suit_elec = pd.merge(suitability_agg, elec_melt, on='fipsstco', how='outer')
suit_elec.to_pickle('suit_elec.pkl')

In [44]:
suit_elec = pd.read_pickle('suit_elec.pkl')

### Median household income and population
Income: American Community Survey 5-year estimates, 2010. <br>
Population: U.S. Census estimates 2010. <br> 
Both from American Fact Finnder,  https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml.

In [46]:
income = pd.read_excel('Med_inc_est10all.xls')
income['fipsstco'] = income.fipsstco.astype(float)
income['med_income'] = income[['Median Household Income']].apply(pd.to_numeric, errors = 'coerce')
income = income.drop(['Median Household Income'], axis=1)

In [50]:
suit_inc = pd.merge(suit_elec, income, on='fipsstco', how='outer')

In [52]:
pop = pd.read_excel('PopulationEst2010.xls')
pop['fipsstco']= pop['FIPS'].astype(float)
pop = pop[['CENSUS_2010_POP', 'fipsstco']].rename(columns = {'CENSUS_2010_POP':'census2010'})

In [53]:
df_final = suit_inc.merge(pop, on='fipsstco', how='left')
df_final.to_pickle('suit_final.pkl')

In [56]:
f_df = pd.read_pickle('suit_final.pkl')

### Direct normal irradiance
The DNI and county geography datasets are both from NSRDB. The DNI is the "Multi Year PSM Direct Normal Irradiance" dataset, which contains annual averages. 

In [59]:
county_geo = gpd.read_file('nrel-us_counties_reatlas.json')

In [60]:
county_geo['fips'] = county_geo['fips'].astype(str).astype(float)
county_geo.to_pickle('county_geo.pkl')

In [67]:
county_geo.rename(columns={'fips':'fipsstco'}, inplace=True)

In [61]:
dni = gpd.read_file('nrel-nsrdb_v3_0_1_1998_2016_dni.json')

In [62]:
county_dni = gpd.sjoin(dni, county_geo, how='inner', op='within')
county_dni = county_dni[['dni', 'fips']]
simple_dni = county_dni.groupby('fips')
agg_dni = simple_dni.agg({'dni':'mean'}).reset_index()

In [63]:
county_geodni = county_geo.merge(agg_dni, on='fips', how='inner')
county_geodni.to_pickle('county_geo_dni.pkl')

In [64]:
county_geo = pd.read_pickle('county_geo_dni.pkl')

In [65]:
county_geo = county_geo[(county_geo.state_name != 'Alaska') & (county_geo.state_name != 'Hawaii') & (county_geo.state_name != 'Puerto Rico')]

### Final merges
County suitability (elections, population, income, prices, small buildings) merged with DNI/county geometry data, then merged with solar installation from OpenPV. 

In [68]:
county_df = county_geo.merge(df_final, on='fipsstco', how='left')

In [69]:
solar2 = pd.read_pickle('solar.pkl')

#limit OpenPV data to useful columns for county analysis and restrict to 2000-2015
solar_short = solar2[['date_installed', 'type', 'st_cnty', 'rl_cost', 'rl_cost_per_watt', 'installer']]
solar_county= solar_short.loc[(solar_short['date_installed'] >= '2000-01-01') & (solar_short['date_installed'] <= '2015-12-31')]
#and remove cost outliers before calculating means
solar_county = solar_county[solar_county.rl_cost <=100000] 
#group and aggregate OpenPV data by county
solar_county_grp = solar_county.groupby('st_cnty').agg({'type':'count', 'rl_cost': 'mean', 'rl_cost_per_watt':'mean', 'installer': lambda x: pd.Series.nunique(x)}).rename(columns= {'type':'installs'}).reset_index()

In [70]:
pv_county = pd.merge(county_df, solar_county_grp, on='st_cnty', how='left')

C:\Users\Sarah\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1481: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  rlab = rizer.factorize(rk)


In [71]:
pv_county['pv_bld'] = (pv_county.installs/(pv_county.nbld/1000))

In [72]:
pv_county.describe()

,gid,fipsstco,area,dni,pct_suitab,rl_price,nbld,pct_rep_2004,pct_rep_2008,pct_rep_2012,...,pct_oth_2004,pct_oth_2008,pct_oth_2012,med_income,census2010,rl_cost,installs,rl_cost_per_watt,installer,pv_bld
count,3270.00,3270.00,3270.00,3270.00,3199.00,3199.00,3199.00,3070.00,3070.00,3070.00,...,3070.00,3070.00,3070.00,3270.00,3269.00,984.00,984.00,984.00,984.00,984.00
mean,1584.19,30288.76,969.99,5.02,0.83,7.94,26877.27,60.42,57.05,59.97,...,1.00,1.65,1.79,42882.31,97471.56,25996.21,677.19,5.16,18.61,5.55
std,882.65,14761.40,1291.70,0.72,0.06,0.99,72323.47,12.47,13.71,14.57,...,0.64,1.03,0.97,10504.79,306543.95,10955.64,3809.31,1.69,68.01,13.16
min,6.00,1001.00,23.17,3.31,0.42,4.47,0.00,9.30,6.50,5.98,...,0.00,0.00,0.00,20577.00,82.00,3208.17,1.00,0.85,0.00,0.00
25%,823.25,19055.50,443.50,4.56,0.79,7.21,4108.00,52.52,48.04,50.84,...,0.60,1.05,1.16,36064.00,11526.00,18922.91,2.00,4.10,0.00,0.13
50%,1616.50,29119.00,631.28,4.94,0.82,8.11,9239.00,60.90,57.42,61.01,...,0.85,1.58,1.68,41065.00,26175.00,24813.09,8.00,4.87,1.00,0.43
75%,2330.75,45020.50,936.43,5.21,0.86,8.68,21818.50,69.00,67.00,70.41,...,1.30,2.08,2.34,47490.00,67531.00,30640.29,86.25,5.95,9.00,3.64
max,3114.00,56045.00,20106.03,7.84,1.00,10.75,2112205.00,92.00,92.64,95.86,...,13.40,34.76,14.87,119075.00,9818605.00,99112.67,63691.00,18.16,1110.00,85.32


In [73]:
#save full information to avoid having to redo it
pv_county.to_pickle('county_data.pkl')
# save a smaller version that will load faster for analysis
counties = pv_county[['name', 'state_name', 'geometry', 'dni', 'nbld', 'installs', 
                      'rl_cost_per_watt', 'census2010', 'pct_dem_2004']]
counties.to_pickle('county_map.pkl')